# 3.3 推理模型 vs 常规模型

本笔记本将对比推理模型(Reasoning Models)和常规模型的区别,帮助你了解何时使用推理模型。

推理模型(如 DeepSeek R1, OpenAI o1)会在生成答案前进行“思考”,擅长复杂推理任务。

**预估成本**: ~$0.05

**安装依赖**:
```bash
pip install openai
```

In [ ]:
# Install dependencies
!pip install openai -q

In [ ]:
import os
from openai import OpenAI
import json
from typing import Dict, Any, Optional
import time

# Initialize客户端
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# DeepSeek API (兼容 OpenAI 格式)
deepseek_client = OpenAI(
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
)

# 检查 API keys
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  警告: 未设置 OPENAI_API_KEY 环境变量")
else:
    print("✓ OpenAI API key 已设置")

if not os.environ.get("DEEPSEEK_API_KEY"):
    print("⚠️  警告: 未设置 DEEPSEEK_API_KEY 环境变量")
    print("  可在 https://platform.deepseek.com/ 免费获取")
else:
    print("✓ DeepSeek API key 已设置")

In [ ]:
def call_regular_model(prompt: str, model: str = "gpt-4o-mini") -> Dict[str, Any]:
    """调用常规模型"""
    start_time = time.time()
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0  # 使用确定性输出以便比较
    )
    
    elapsed = time.time() - start_time
    
    return {
        "model": model,
        "content": response.choices[0].message.content,
        "reasoning": None,  # 常规模型没有推理过程
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        "time_seconds": elapsed
    }

def call_reasoning_model(prompt: str, model: str = "deepseek-reasoner") -> Dict[str, Any]:
    """调用推理模型 (DeepSeek R1)"""
    start_time = time.time()
    
    response = deepseek_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    elapsed = time.time() - start_time
    
    # DeepSeek R1 在 reasoning_content 字段返回思考过程
    reasoning = None
    if hasattr(response.choices[0].message, 'reasoning_content'):
        reasoning = response.choices[0].message.reasoning_content
    
    return {
        "model": model,
        "content": response.choices[0].message.content,
        "reasoning": reasoning,
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        "time_seconds": elapsed
    }

def compare_models(prompt: str, description: str = ""):
    """对比常规模型和推理模型"""
    print("=" * 80)
    if description:
        print(f"测试场景: {description}")
    print(f"\n提示词: {prompt}")
    print("=" * 80)
    
    # Test常规模型
    print("\n🤖 常规模型 (GPT-4o-mini)")
    print("-" * 80)
    try:
        regular_result = call_regular_model(prompt)
        print(f"\n{regular_result['content']}")
        print(f"\n📊 Tokens: {regular_result['usage']['total_tokens']} | "
              f"时间: {regular_result['time_seconds']:.2f}s")
    except Exception as e:
        print(f"❌ 错误: {e}")
        regular_result = None
    
    # Test推理模型
    print("\n" + "=" * 80)
    print("\n🧠 推理模型 (DeepSeek R1)")
    print("-" * 80)
    try:
        reasoning_result = call_reasoning_model(prompt)
        
        # 显示思考过程(如果有)
        if reasoning_result['reasoning']:
            print("\n💭 思考过程:")
            # 只显示前500字符,避免过长
            reasoning_preview = reasoning_result['reasoning'][:500]
            print(f"{reasoning_preview}..." if len(reasoning_result['reasoning']) > 500 
                  else reasoning_result['reasoning'])
            print(f"\n[完整思考过程: {len(reasoning_result['reasoning'])} 字符]")
        
        print("\n✅ 最终答案:")
        print(f"{reasoning_result['content']}")
        print(f"\n📊 Tokens: {reasoning_result['usage']['total_tokens']} | "
              f"时间: {reasoning_result['time_seconds']:.2f}s")
    except Exception as e:
        print(f"❌ 错误: {e}")
        reasoning_result = None
    
    print("\n" + "=" * 80)
    return regular_result, reasoning_result

## Demo 1: 数学推理问题

推理模型在多步骤数学问题上表现更好。

In [ ]:
math_problem = """
小明有 100 元,他买了 3 本书,每本 15 元。
然后他用剩余钱的一半买了零食。
之后他又赚了 20 元。
最后他买了一个 30 元的玩具。

请问小明现在还剩多少钱?
请详细列出计算步骤。
"""

compare_models(math_problem, "多步骤数学计算")

## Demo 2: 查看推理Models的思考过程
<!-- Demo 2: 查看推理模型的思考过程 -->

推理模型的独特之处在于它会暴露“思考”过程,让我们看看它如何分析问题。

In [ ]:
reasoning_problem = """
有三个开关和三盏灯,开关在楼下,灯在楼上。
你只能上楼一次,如何确定哪个开关控制哪盏灯?
"""

print("查看推理模型的完整思考过程\n")
print("=" * 80)
print(f"问题: {reasoning_problem}")
print("=" * 80)

try:
    result = call_reasoning_model(reasoning_problem)
    
    if result['reasoning']:
        print("\n🧠 完整思考过程:")
        print("-" * 80)
        print(result['reasoning'])
        print("-" * 80)
        print(f"\n思考过程长度: {len(result['reasoning'])} 字符")
    
    print("\n" + "=" * 80)
    print("\n✅ 最终答案:")
    print("-" * 80)
    print(result['content'])
    
    print("\n" + "=" * 80)
    print("\n💡 观察:")
    print("- 推理模型会先分析问题,考虑多种可能性")
    print("- 思考过程展示了问题分解和逻辑推演")
    print("- 最终答案更有条理和依据")
    
except Exception as e:
    print(f"❌ 错误: {e}")
    print("\n提示: 确保已设置 DEEPSEEK_API_KEY 环境变量")

## Demo 3: 逻辑谜题Comparison
<!-- Demo 3: 逻辑谜题对比 -->

测试一个需要复杂逻辑推理的谜题,看常规模型和推理模型的表现差异。

In [ ]:
logic_puzzle = """
有三个人:A、B、C。
- A 说:“B 在说谎。”
- B 说:“C 在说谎。”
- C 说:“A 和 B 都在说谎。”

已知这三个人中只有一个人说的是真话。
请问谁说的是真话?请给出详细推理过程。
"""

regular_result, reasoning_result = compare_models(logic_puzzle, "逻辑谜题")

print("\n📝 对比分析:")
print("-" * 80)

if regular_result and reasoning_result:
    print("\n常规模型:")
    print(f"  - Token 使用: {regular_result['usage']['total_tokens']}")
    print(f"  - 响应时间: {regular_result['time_seconds']:.2f}s")
    print(f"  - 推理过程: 不可见")
    
    print("\n推理模型:")
    print(f"  - Token 使用: {reasoning_result['usage']['total_tokens']}")
    print(f"  - 响应时间: {reasoning_result['time_seconds']:.2f}s")
    has_reasoning = reasoning_result['reasoning'] is not None
    print(f"  - 推理过程: {'可见 (' + str(len(reasoning_result['reasoning'])) + ' 字符)' if has_reasoning else '不可见'}")
    
    if reasoning_result['usage']['total_tokens'] > regular_result['usage']['total_tokens']:
        ratio = reasoning_result['usage']['total_tokens'] / regular_result['usage']['total_tokens']
        print(f"\n推理模型使用了 {ratio:.1f}x 的 tokens (因为包含思考过程)")

## Demo 4: 何时不应该使用推理Models
<!-- Demo 4: 何时不应该使用推理模型 -->

推理模型更慢、更贵,对简单任务是过度设计。让我们测试几个简单任务。

In [ ]:
simple_tasks = [
    {
        "task": "简单翻译",
        "prompt": "将以下英文翻译成中文: Hello, how are you today?"
    },
    {
        "task": "基础分类",
        "prompt": "判断以下评论的情感(正面/负面/中性): 这个产品还不错,但价格有点贵。"
    },
    {
        "task": "简单提取",
        "prompt": "从以下句子提取人名: 李雷和韩梅梅是好朋友。"
    }
]

print("推理模型的不适用场景\n")
print("=" * 80)

for task_info in simple_tasks:
    print(f"\n任务类型: {task_info['task']}")
    print(f"提示词: {task_info['prompt']}")
    print("-" * 80)
    
    # 常规模型
    try:
        regular = call_regular_model(task_info['prompt'])
        print(f"\n常规模型: {regular['content'][:100]}")
        print(f"  ⚡ {regular['time_seconds']:.2f}s | {regular['usage']['total_tokens']} tokens")
    except Exception as e:
        print(f"常规模型错误: {e}")
        regular = None
    
    # 推理模型
    try:
        reasoning = call_reasoning_model(task_info['prompt'])
        print(f"\n推理模型: {reasoning['content'][:100]}")
        print(f"  🐌 {reasoning['time_seconds']:.2f}s | {reasoning['usage']['total_tokens']} tokens")
    except Exception as e:
        print(f"推理模型错误: {e}")
        reasoning = None
    
    if regular and reasoning:
        time_ratio = reasoning['time_seconds'] / regular['time_seconds']
        token_ratio = reasoning['usage']['total_tokens'] / regular['usage']['total_tokens']
        print(f"\n  📊 推理模型 vs 常规模型:")
        print(f"     - 时间: {time_ratio:.1f}x 更慢")
        print(f"     - Tokens: {token_ratio:.1f}x 更多")
        print(f"     - 价值: 对于简单任务,额外成本和时间不值得")
    
    print("\n" + "=" * 80)

print("\n💡 结论:")
print("-" * 80)
print("对于简单、直接的任务(翻译、分类、提取),常规模型:")
print("  ✓ 响应更快")
print("  ✓ 成本更低")
print("  ✓ 输出质量足够好")
print("\n只在真正需要复杂推理时才使用推理模型!")

## 推理Models使用指南
<!-- 推理模型使用指南 -->

总结何时应该(和不应该)使用推理模型。

In [ ]:
import pandas as pd

# Create对比表
comparison_table = pd.DataFrame([
    {
        "任务类型": "多步数学问题",
        "推荐模型": "推理模型",
        "原因": "需要计划和验证计算步骤",
        "示例": "复杂的应用题、多步计算"
    },
    {
        "任务类型": "逻辑谜题",
        "推荐模型": "推理模型",
        "原因": "需要假设、推演和验证",
        "示例": "数独、逻辑推理题"
    },
    {
        "任务类型": "代码调试",
        "推荐模型": "推理模型",
        "原因": "需要分析因果关系",
        "示例": "复杂 bug 分析、算法优化"
    },
    {
        "任务类型": "策略规划",
        "推荐模型": "推理模型",
        "原因": "需要权衡利弊、考虑多种方案",
        "示例": "系统架构设计、业务决策"
    },
    {
        "任务类型": "科学推理",
        "推荐模型": "推理模型",
        "原因": "需要基于证据得出结论",
        "示例": "实验分析、因果推断"
    },
    {
        "任务类型": "简单翻译",
        "推荐模型": "常规模型",
        "原因": "直接映射,无需推理",
        "示例": "文本翻译、格式转换"
    },
    {
        "任务类型": "情感分类",
        "推荐模型": "常规模型",
        "原因": "模式识别,不需要思考",
        "示例": "评论分类、情绪检测"
    },
    {
        "任务类型": "信息提取",
        "推荐模型": "常规模型",
        "原因": "结构化提取,无需推理",
        "示例": "实体识别、关键词提取"
    },
    {
        "任务类型": "文本改写",
        "推荐模型": "常规模型",
        "原因": "语言转换,不需要分析",
        "示例": "润色、缩写、扩写"
    },
    {
        "任务类型": "简单问答",
        "推荐模型": "常规模型",
        "原因": "事实检索,无需推演",
        "示例": "定义解释、基础知识"
    },
])

print("\n推理模型 vs 常规模型使用指南\n")
print(comparison_table.to_string(index=False))

print("\n\n" + "=" * 80)
print("\n🎯 决策流程:")
print("-" * 80)
print("""
1. 问自己:这个任务需要“思考”吗?
   - 需要多步推理? → 推理模型
   - 需要权衡选项? → 推理模型
   - 需要验证答案? → 推理模型
   - 只是简单转换? → 常规模型

2. 考虑成本和时间:
   - 推理模型通常 2-5x 更慢
   - 推理模型通常 2-10x 更贵(因为思考 tokens)
   - 确保额外成本物有所值

3. 先用常规模型测试:
   - 如果常规模型表现好,就不需要推理模型
   - 只在常规模型失败或不够准确时升级

4. 监控效果:
   - 记录准确率提升
   - 计算成本收益比
   - 决定是否值得长期使用推理模型
""")

print("=" * 80)
print("\n💰 成本对比(2026年2月估算):")
print("-" * 80)
print("""
常规模型 (GPT-4o-mini):
  - 输入: $0.15 / 1M tokens
  - 输出: $0.60 / 1M tokens
  - 速度: 快 (~1-2s)

推理模型 (DeepSeek R1):
  - 输入: $0.55 / 1M tokens
  - 输出: $2.19 / 1M tokens
  - 缓存输出: $1.09 / 1M tokens
  - 速度: 较慢 (~3-10s)
  - 注意: 思考过程也计入 token 数!

OpenAI o1 系列:
  - 更强但更贵
  - 适合最复杂的推理任务
""")

## 练习: 找一个适合推理Models的问题
<!-- 练习: 找一个适合推理模型的问题 -->

思考你工作中遇到的问题,哪些可以从推理模型中受益?

In [ ]:
# 在这里输入你的问题
your_problem = """
在这里描述一个你工作中遇到的复杂问题...

例如:
- 需要多步骤分析的技术决策
- 需要权衡多个因素的架构选择
- 复杂的数据分析或推理任务
- 需要发现隐藏模式或因果关系的问题
"""

if your_problem.strip() and "在这里描述" not in your_problem:
    print("分析你的问题...\n")
    print("=" * 80)
    
    # 用常规模型分析这个问题是否适合推理模型
    analysis_prompt = f"""
分析以下问题是否适合使用推理模型(如 DeepSeek R1, OpenAI o1):

{your_problem}

请评估:
1. 这个问题是否需要多步推理?
2. 是否需要权衡多个选项?
3. 是否需要发现隐藏的因果关系?
4. 常规模型可能在哪里失败?
5. 推理模型能带来什么价值?

最后给出明确建议:适合/不适合推理模型,以及理由。
"""
    
    try:
        analysis = call_regular_model(analysis_prompt, model="gpt-4o-mini")
        print("📊 适用性分析:")
        print("-" * 80)
        print(analysis['content'])
        print("\n" + "=" * 80)
        
        # 如果判断适合,可以尝试用推理模型解决
        print("\n要用推理模型尝试解决这个问题吗? (y/n)")
        print("提示: 取消注释下面的代码并运行")
        
        # # 取消下面的注释来运行
        # try:
        #     reasoning_solution = call_reasoning_model(your_problem)
        #     print("\n🧠 推理模型的思考过程:")
        #     if reasoning_solution['reasoning']:
        #         print(reasoning_solution['reasoning'][:1000] + "...")
        #     print("\n✅ 推理模型的答案:")
        #     print(reasoning_solution['content'])
        # except Exception as e:
        #     print(f"错误: {e}")
        
    except Exception as e:
        print(f"分析失败: {e}")
else:
    print("请在上面的 your_problem 变量中输入你的问题,然后重新运行此单元格。")
    print("\n💡 提示: 选择一个真正需要'思考'的复杂问题")

## 总结

通过本笔记本的实验,你应该理解了:

### 1. 推理模型的特点

**优势**:
- 在复杂推理任务上表现更好
- 可以展示思考过程,提高可解释性
- 能够进行多步骤的逻辑推演和验证
- 在数学、逻辑、策略规划等任务上更准确

**劣势**:
- 响应时间更长(2-5x)
- Token 消耗更多(包含思考过程)
- 成本更高(2-10x)
- 对简单任务是过度设计

### 2. 何时使用推理模型

**适合的场景**:
- ✓ 多步数学问题和应用题
- ✓ 逻辑谜题和推理题
- ✓ 复杂代码调试和算法分析
- ✓ 需要权衡的策略决策
- ✓ 科学推理和因果分析

**不适合的场景**:
- ✗ 简单翻译和转换
- ✗ 情感分类和实体识别
- ✗ 直接的信息提取
- ✗ 基础问答和知识检索
- ✗ 文本润色和改写

### 3. 最佳实践

1. **先用常规模型**: 如果常规模型够用,就不需要推理模型
2. **评估成本收益**: 确保额外成本能带来相应的价值提升
3. **利用思考过程**: 推理模型的思考过程本身就是有价值的输出
4. **监控准确率**: 量化推理模型带来的改进
5. **混合使用**: 简单任务用常规模型,复杂任务用推理模型

### 4. 可用的推理模型

- **DeepSeek R1**: 开源,成本较低,性能强大
- **OpenAI o1**: 最强但最贵,适合最复杂的任务
- **OpenAI o1-mini**: o1 的轻量版,平衡性能和成本

记住:**推理能力是有代价的**,只在真正需要时使用!